In [ ]:
from VR_Trajectory_analysis import *

In [ ]:
directory = '/Users/apaula/ownCloud/MatrexVR1/20241203_TM_Ants_Grey_patch1_Data/RunData'

In [ ]:
df = get_combined_df(directory, trim_seconds=1.0)

In [ ]:
df = add_trial_id_and_displacement(df)
df = add_trial_time(df)

In [ ]:
#remove trials, because they where from a keyboard test
df = df[~df['UniqueTrialID'].isin([0, 1, 2, 3])]

In [ ]:
df_stationary, df_normal, df_excessive, stationary_ids, normal_ids, excessive_ids = classify_trials_by_displacement(df[df['Scene']=='Choice_noBG'], min_disp=0, max_disp=500)

In [ ]:
df_stationary, df_normal, df_excessive, stationary_ids, normal_ids, excessive_ids = classify_trials_by_displacement(df[df['Scene']=='Choice_noBG'], min_disp=60, max_disp=500)

In [ ]:
forrest_stationary, forrest_normal, forrest_excessive, stationary_ids, normal_ids, excessive_ids = classify_trials_by_displacement(df[df['Scene']=='Forrest_choice'], min_disp=14, max_disp=200)

In [ ]:
plot_trajectories(df_stationary, 'Stationary')

In [ ]:
plot_trajectories(df_normal[df_normal['UniqueTrialID']], 'Normal') 

In [ ]:
plot_trajectories(df_excessive, 'Stationary') 

In [ ]:
df_normal

In [ ]:
import pandas as pd
import numpy as np

# Define goal coordinates (same as before)
center_goal = (0, 60)
left_goal = (-10.416, 59.088)
right_goal = (10.416, 59.088)

# Threshold for reaching the goal
threshold = 4  # 4 cm

# Config files that have only the center goal
center_only_configs = [
    "BinaryChoice10_BlackCylinder_control.json",
    "BinaryChoice10_constantSize_BlackCylinder_control.json"
]

def distance(p1, p2):
    return np.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

# Assuming df is your DataFrame
# Make sure the data is sorted by time if not already
# df = df.sort_values(by='trial_time')

results = []

for trial_id, trial_data in df_normal.groupby('UniqueTrialID'):
    config = trial_data['ConfigFile'].iloc[0]
    
    # Determine relevant goals
    if config in center_only_configs:
        goals = [('center', center_goal)]
    else:
        goals = [('left', left_goal), ('right', right_goal)]
    
    first_reached = None
    reached_time = None
    
    # Ensure data is time-sorted (if not done globally)
    trial_data = trial_data.sort_values(by='trial_time')
    
    for idx, row in trial_data.iterrows():
        participant_pos = (row['GameObjectPosX'], row['GameObjectPosZ'])
        
        # Check each goal
        for goal_name, goal_pos in goals:
            dist = distance(participant_pos, goal_pos)
            if dist <= threshold:
                first_reached = goal_name
                reached_time = row['trial_time']  # store the time at which the goal was reached
                break
        
        if first_reached is not None:
            break
    
    results.append((trial_id, first_reached, reached_time))

results_df = pd.DataFrame(results, columns=['UniqueTrialID', 'FirstReachedGoal', 'GoalReachedTime'])

print(results_df)


In [ ]:
results_df.to_pickle("results_df.pkl")

In [ ]:
results_df = pd.read_pickle("results_df.pkl")

In [ ]:
results_df

In [ ]:

# Example: Get the trajectory for a specific trial only until the goal is reached
target_trial_id = 12  # example
trial_info = results_df.loc[results_df['UniqueTrialID'] == target_trial_id]

if not trial_info.empty:
    cutoff_time = trial_info['GoalReachedTime'].iloc[0]
    trial_trajectory = df[(df['UniqueTrialID'] == target_trial_id) & (df['trial_time'] <= cutoff_time)]
    # Now 'trial_trajectory' contains only the portion of the trial before the goal was reached.
    plot_trajectories(trial_trajectory, 'Normal') 

In [ ]:
import numpy as np
import pandas as pd

# 1. Keep only trials that actually reached a goal
valid_results = results_df.dropna(subset=['FirstReachedGoal'])

# 2. Merge the cutoff times back into df
#    We merge on 'UniqueTrialID' to get each trial's GoalReachedTime.
df_merged = pd.merge(df, valid_results[['UniqueTrialID', 'GoalReachedTime']], on='UniqueTrialID', how='inner')

# 3. Filter df so that only rows with trial_time less than or equal to the goal time are kept
df_cut = df_merged[df_merged['trial_time'] <= df_merged['GoalReachedTime']]

# Now df_cut contains only the subsets of trials that reached a goal, truncated at the goal reach time.

In [ ]:
df_cut['ConfigFile'].unique()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assume these from previous steps:
center_only_configs = [
    "BinaryChoice10_BlackCylinder_control.json",
    "BinaryChoice10_constantSize_BlackCylinder_control.json"
]

# Identify which ConfigFiles have multiple goals
# Since we know only the two listed above are single-goal configs,
# all others are multi-goal configs.
all_configs = df_cut['ConfigFile'].unique()
multi_goal_configs = [c for c in all_configs if c not in center_only_configs]

# Merge results_df to get the goal reached information into df_cut if needed
# (Not strictly necessary if we only need ratios. We can just use results_df separately.)
# But let's have a convenient DataFrame for ratio calculations.
df_joined = pd.merge(df_cut, results_df[['UniqueTrialID', 'FirstReachedGoal']], on='UniqueTrialID', how='left')

# Group the truncated data by ConfigFile
for config, group in df_joined.groupby('ConfigFile'):
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Plot each trial’s trajectory
    for trial_id, trial_data in group.groupby('UniqueTrialID'):
        ax.plot(trial_data['GameObjectPosX'], trial_data['GameObjectPosZ'], alpha=0.3)
    
    ax.set_ylim(-10, 70)
    ax.set_xlim(-40, 40)
    
    # Make axes equal
    ax.set_aspect('equal', adjustable='box')

    # Set plot titles and labels
    ax.set_title(f"Trajectories for ConfigFile: {config}")
    ax.set_xlabel("X Position (cm)")
    ax.set_ylabel("Z Position (cm)")
    ax.grid(True)
    
    # If it's a multi-goal config, calculate ratio left/right
    if config in multi_goal_configs:
        # Filter the trials for this config in results_df
        config_results = results_df.loc[results_df['UniqueTrialID'].isin(group['UniqueTrialID'].unique())]
        
        # Count how many reached left vs right
        left_count = (config_results['FirstReachedGoal'] == 'left').sum()
        right_count = (config_results['FirstReachedGoal'] == 'right').sum()
        
        # Compute ratio (e.g., left/total and right/total)
        total = left_count + right_count
        if total > 0:
            left_ratio = left_count / total
            right_ratio = right_count / total
            ratio_text = f"Left: {left_count} ({left_ratio:.2f}), Right: {right_count} ({right_ratio:.2f})"
        else:
            ratio_text = "No goals reached"
        
        # Add text box with ratio information
        ax.text(0.05, 0.95, ratio_text, transform=ax.transAxes, va='top', bbox=dict(boxstyle="round", fc="w", ec="0.5"))
    
    elif config in center_only_configs:
        # Filter the trials for this config in results_df
        config_results = results_df.loc[results_df['UniqueTrialID'].isin(group['UniqueTrialID'].unique())]
        
        # Count how many reached left vs right
        count = (config_results['FirstReachedGoal'] == 'center').sum()
        # Only center goal, no ratio needed
        ax.text(0.05, 0.95, f"Center goal only: {count}", transform=ax.transAxes, va='top', bbox=dict(boxstyle="round", fc="w", ec="0.5"))
    
    
    output_dir = '/Users/apaula/Downloads'
    # Remove '.json' from config filename and prepend 'trajectories_'
    base_name = os.path.splitext(config)[0]
    filename = f"trajectories_{base_name}.png"
    filepath = os.path.join(output_dir, filename)
    
    # Save the figure
    fig.savefig(filepath, dpi=300)

    plt.tight_layout()
    plt.show()


In [ ]:
import pandas as pd

# Suppose you have `df` that contains columns: UniqueTrialID, ConfigFile, ...
# and `results_df` that contains UniqueTrialID, FirstReachedGoal, GoalReachedTime

# First, merge results_df with df to get ConfigFile in results_df
results_with_config = pd.merge(results_df, df[['UniqueTrialID', 'ConfigFile']], on='UniqueTrialID', how='left')

# Define sets for easier logic
all_black_configs = {
    "BinaryChoice10_BlackCylinder_control.json",
    "BinaryChoice10_constantSize_BlackCylinder_control.json",
    "BinaryChoice11_BlackCylinder_BlackCylinder.json",
    "BinaryChoice11_constantSize_BlackCylinder_BlackCylinder.json"
}

left_grey_right_black_configs = {
    "BinaryChoice31_323232_BlackCylinder.json",
    "BinaryChoice31_constantSize_323232_BlackCylinder.json"
}

left_black_right_grey_configs = {
    "BinaryChoice13_BlackCylinder_323232.json",
    "BinaryChoice13_constantSize_BlackCylinder_323232.json"
}

def determine_goal_type(config, reached_goal):
    # If no goal reached, return None
    if pd.isna(reached_goal):
        return None
    
    # Configs where all goals are black
    if config in all_black_configs:
        return "black"
    
    # Configs where left=grey, right=black
    if config in left_grey_right_black_configs:
        if reached_goal == "left":
            return "grey"
        elif reached_goal == "right":
            return "black"
        elif reached_goal == "center":  # If center exists, treat as black or grey?
            # Center isn't defined for these configs, likely doesn't exist
            return None
    
    # Configs where left=black, right=grey
    if config in left_black_right_grey_configs:
        if reached_goal == "left":
            return "black"
        elif reached_goal == "right":
            return "grey"
        elif reached_goal == "center":
            # Again, center not defined here, but if occurs:
            return None
    
    # If config doesn't fit into known sets or reached_goal is something unexpected:
    return None

# Apply the logic to create a new column "GoalType"
results_with_config['GoalType'] = results_with_config.apply(
    lambda row: determine_goal_type(row['ConfigFile'], row['FirstReachedGoal']), axis=1
)

# Now results_with_config has an added "GoalType" column
# If you want to save back to results_df (with goal type), just assign:
results_df = results_with_config

# results_df now has: UniqueTrialID, FirstReachedGoal, GoalReachedTime, ConfigFile, GoalType


In [ ]:
results_df

In [ ]:
results_df.to_pickle("results_df2.pkl")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assume these from previous steps:
center_only_configs = [
    "BinaryChoice10_BlackCylinder_control.json",
    "BinaryChoice10_constantSize_BlackCylinder_control.json"
]

# Identify which ConfigFiles have multiple goals
# Since we know only the two listed above are single-goal configs,
# all others are multi-goal configs.
all_configs = df_cut['ConfigFile'].unique()
multi_goal_configs = [c for c in all_configs if c not in center_only_configs]

# Merge results_df to get the goal reached information into df_cut if needed
# (Not strictly necessary if we only need ratios. We can just use results_df separately.)
# But let's have a convenient DataFrame for ratio calculations.
df_joined = pd.merge(df_cut, results_df[['UniqueTrialID', 'FirstReachedGoal']], on='UniqueTrialID', how='left')

# Group the truncated data by ConfigFile
for config, group in df_joined.groupby('ConfigFile'):
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Plot each trial’s trajectory
    for trial_id, trial_data in group.groupby('UniqueTrialID'):
        ax.plot(trial_data['GameObjectPosX'], trial_data['GameObjectPosZ'], alpha=0.3)
    
    ax.set_ylim(-10, 70)
    ax.set_xlim(-40, 40)
    
    # Make axes equal
    ax.set_aspect('equal', adjustable='box')

    # Set plot titles and labels
    ax.set_title(f"Trajectories for ConfigFile: {config}")
    ax.set_xlabel("X Position (cm)")
    ax.set_ylabel("Z Position (cm)")
    ax.grid(True)
    
    # If it's a multi-goal config, calculate ratio left/right
    if config in multi_goal_configs:
        # Filter the trials for this config in results_df
        config_results = results_df.loc[results_df['UniqueTrialID'].isin(group['UniqueTrialID'].unique())]
        
        # Count how many reached left vs right
        left_count = (config_results['FirstReachedGoal'] == 'left').sum()
        right_count = (config_results['FirstReachedGoal'] == 'right').sum()
        
        # Compute ratio (e.g., left/total and right/total)
        total = left_count + right_count
        if total > 0:
            left_ratio = left_count / total
            right_ratio = right_count / total
            ratio_text = f"Left: {left_count} ({left_ratio:.2f}), Right: {right_count} ({right_ratio:.2f})"
        else:
            ratio_text = "No goals reached"
        
        # Add text box with ratio information
        ax.text(0.05, 0.95, ratio_text, transform=ax.transAxes, va='top', bbox=dict(boxstyle="round", fc="w", ec="0.5"))
    
    elif config in center_only_configs:
        # Filter the trials for this config in results_df
        config_results = results_df.loc[results_df['UniqueTrialID'].isin(group['UniqueTrialID'].unique())]
        
        # Count how many reached left vs right
        count = (config_results['FirstReachedGoal'] == 'center').sum()
        # Only center goal, no ratio needed
        ax.text(0.05, 0.95, f"Center goal only: {count}", transform=ax.transAxes, va='top', bbox=dict(boxstyle="round", fc="w", ec="0.5"))
    
    
    output_dir = '/Users/apaula/Downloads'
    # Remove '.json' from config filename and prepend 'trajectories_'
    base_name = os.path.splitext(config)[0]
    filename = f"trajectories_{base_name}.png"
    filepath = os.path.join(output_dir, filename)
    
    # Save the figure
    fig.savefig(filepath, dpi=300)

    plt.tight_layout()
    plt.show()
